## Task 1

$$E\lbrack B_{r}\rbrack\  = \ S_{T}^{- 1}S\beta^{*}$$

Where $S\  = \ X^{T}X$ , $S_{T}\  = \ X^{T}X\  + \ \tau I$

$$X^{T}\  = \ V\lambda U^{t}$$

$$X\  = \ U\lambda V^{T}$$

$$X^{T}X\  = \ V\lambda^{2}V^{T}$$

For, $E\lbrack\beta_{r}\rbrack\  = \ S_{T}^{- 1}S\beta^{*}$

= ${(X^{T}X\  + \ \tau I)}^{- 1}\ (X^{T}X)\beta^{*}$

= $V{(\lambda^{2} + \tau I)}^{- 1}\ V^{T}V\lambda^{2}U^{T}\beta^{*}$

= $V{(\lambda^{2} + \tau I)}^{- 1}\ \lambda^{2}U^{T}\beta^{*}$

= $V\lambda^{|}U^{T}\beta^{*}$

= ${(X^{T}X\  + \ \tau I)}^{- 1}X^{T}X\beta$

= $S_{T}^{- 1}S\beta^{*}$

For $cov\lbrack\beta_{r}\rbrack\  = \ S_{T}^{- 1}SS_{T}^{- 1}\sigma^{2}$

=
${(X^{T}X\  + \ \tau I)}^{- 1}\ (X^{T}X)\ {(X^{T}X\  + \ \tau I)}^{- 1}\ \sigma^{2}$

=
$V{(\lambda^{2} + \tau I)}^{- 1}\ V^{T}V\lambda^{2}U^{T}(V{(\lambda^{2} + \tau I)}^{- 1})\ V^{T}$

=
$V{(\lambda^{2} + \tau I)}^{- 1}\ \lambda^{2}U^{T}(V{(\lambda^{2} + \tau I)}^{- 1})\ V^{T}$

=
$V\lambda^{|}U^{T}\ V\lambda^{|}U^{T}V{\lambda^{2}}^{|}U^{T}\sigma^{2}$

= $S_{T}^{- 1}SS_{T}^{- 1}\sigma^{2}$


In [21]:
from warnings import filters
import numpy as np
from os import path
from scipy.sparse import coo_matrix, csc_matrix
from scipy.ndimage import filters
from scipy.sparse.linalg import lsqr
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 3 automatic feature selection for LDA as regression

In [22]:
# import data

data_path = "../04/"
M, alphas, y = 195, np.load(path.join(data_path, "hs_tomography/alphas_195.npy")), np.load(path.join(data_path, "./hs_tomography/y_195.npy"))
Np = 275
# append D zeros to y
D = np.square(M)
y_ = np.append(y, np.zeros((1, D)))
print(y.size)


49225


In [23]:
def construct_X(M, alphas, Np = None):
    D = M*M
    # define sensor size
    if Np is None:
        Np = int(np.ceil(np.sqrt(2) * M))
        if Np % 2 == 0: Np += 1
    # number of angles
    No = len(alphas)

    # flattened output coordinates
    j = np.mgrid[0:D].astype(np.int32)
    # coordinate matrix for the output pixels
    M2 = (M-1) / 2
    grid = np.mgrid[-M2:M-M2,-M2:M-M2].swapaxes(1,2).reshape(2,D)

    # collect indices and corresponding values for all iterations
    i_indices = []
    j_indices = []
    weights = []

    for k, alpha in enumerate(alphas):
        # convert angle and prepare projection vector
        alph_rad = np.radians(alpha)
        proj_vec = np.array([np.cos(alph_rad), -np.sin(alph_rad)])
        # project coordinates
        proj = np.dot(proj_vec, grid) + Np // 2
        # compute sensor indices and weights below the projected points
        i = np.floor(proj)
        w = (i+1) - proj
        # make sure rays falling outside the sensor are not counted
        clip = np.logical_and(0 <= i, i < Np-1)
        i_indices.append((i + k*Np)[clip])
        j_indices.append(j[clip])
        weights.append(w[clip])
        # compute sensor indices and weights above the projected points
        w = proj - i
        i_indices.append((i+1 + k*Np)[clip])
        j_indices.append(j[clip])
        weights.append(w[clip])

    # construct matrix X
    i = np.concatenate(i_indices).astype(np.int32)
    j = np.concatenate(j_indices).astype(np.int32)
    w = np.concatenate(weights)
    X = coo_matrix((w, (i,j)), shape = (No*Np, D), dtype = np.float32)
    return X

## 3.1 Implement Orthogonal Matching Pursuit

In [37]:
def omp_regression(X, y, T):
    N=X.shape[0]
    D=X.shape[1]
    
    A_0=[]
    B_0=list(range(0,D))
    r=y.copy()
    
    beta_t_Hat=[]
    X_t_B=X.copy()
    
    for t in range(1,T+1):
        j_vec=np.linalg.norm(np.transpose(X)*r,axis=1)
        j_test=np.sort(j_vec)[::-1]
        for test in j_test:
            vector_to_check = np.where(j_vec==test)
            if(vector_to_check[0].shape[0]==1):
                if vector_to_check[0] in B_0:
                    j=np.where(j_vec==test)[0]
                    break
                else:
                    continue
            else:
                found_j=-1
                for possible_j in vector_to_check[0]:
                    if possible_j in B_0:
                        found_j=possible_j
                        break
                    else:
                        continue
                        
                if found_j==-1:
                    continue
                else:
                    j=found_j
                    break
                        
        j_t=B_0.pop(B_0.index(j))
        A_0.append(j_t)
        X_t=X[:,A_0]
        #X_t_B=X[:,B_0]
        beta_t,residuals,rank,s=np.linalg.lstsq(X_t,y,rcond=None)
        r=residuals
        beta_t_hat=np.zeros(D)
        beta_t_hat[A_0]=beta_t
        beta_t_Hat.append(beta_t_hat)
    return beta_t_Hat

In [44]:
def omp_regression(X, y ,T=99):
    print("X:",X.shape," y: ", y.shape, "T:",T)
    A, B, r, R = np.zeros(T),np.arange(X.shape[1]), y, np.zeros((X.shape[0],T))
    print(A.shape,B.shape,R.shape)
    X = csc_matrix(X)
    print(X.shape)
    for t in range(T):
        j = np.argmax(np.abs(X.T[B]*r))
        A[t] = B[j]
        B = np.delete(B,j)
        Xt = X[:,A]
        βt = lsqr(Xt,y)[0]
        R[t] = βt
        r = y - Xt*βt
    return R

## 3.2 Classification wtih sparse LDA

In [32]:
def load_and_prepare_data():
    # read and prepare the digits data
    digits = load_digits()

    data = digits['data']
    target = digits['target']

    #filter targets and data - to only contain datasets about 1 and 7 
    data_1   =   data[target == 1]
    target_1 = target[target == 1]
    
    data_7   =   data[target == 7]
    target_7 = target[target == 7]
    target_7[:] = -1
    
    data = np.append(data_1, data_7).reshape(-1,64)
    target = np.append(target_1, target_7)
    
    return data,target

In [35]:
def determine_error(results,T,x_test,y_test):
    #print(f"x_test: {x_test.shape}, T: {T}, y_test: {y_test.shape}, results: {results.shape}")
    results_t_i = np.sum(x_test[:]*results[T-1],axis=1)
    results_t_i[results_t_i>=0] =  1
    results_t_i[results_t_i<0]  = -1
    err_t_i = np.where(results_t_i!=y_test)[0].shape[0] / y_test.shape[0]
    print('error rate for T='+str(T)+': '+str(err_t_i) )

In [46]:
X,y = load_and_prepare_data()
x_training, x_test, y_training, y_test = train_test_split(X, y,test_size=0.3,random_state=42)
results = omp_regression(x_training,y_training,54)
print(results[0].shape)
#lets now determine the error rates
#therefore we will use a classifier which computes the results and the error rates

determine_error(results,1 ,x_test,y_test)
determine_error(results,2 ,x_test,y_test)
determine_error(results,4 ,x_test,y_test)
determine_error(results,8 ,x_test,y_test)
determine_error(results,12,x_test,y_test)
determine_error(results,16,x_test,y_test)
determine_error(results,32,x_test,y_test)
determine_error(results,48,x_test,y_test)

print("Starting from T = 16 the error is no longer decreased")

#we want to print a 8 by 8 picture which is all white but the pixels which are turned on are black in increasing 
#intensity shwoing the pixels which are most important in highest identity
importance = np.ones(64)
tendencie  = np.ones(64) 

#we now nee to find the order in which the pixels have been tunred on
helper = np.zeros(64)
indecies_used=[]
for i in range(0,16):
    indecies_changed = np.where(helper!=results[i])[0]
    index_added = -1
    for possible_i in indecies_changed:
        if possible_i in indecies_used:
            continue
        else:
            index_added = possible_i
            indecies_used.append(possible_i)
            break
    
    helper = results[i]
    importance[index_added] = i * (1/32)
    if results[i][index_added]<0:
        tendencie[index_added] = 0.5
    else:
        tendencie[index_added] = 0

print('----------------------------')
print("We used the following pixels:")  
print(indecies_used)
print('----------------------------')
        
importance = importance.reshape(8,8)
tendencie  = tendencie.reshape(8,8)

fig = plt.figure()
imgplot = plt.imshow(importance,cmap="hot")
plt.title('Heatmap for T=16')

##next up we anna show if those pixels point towards one or two
fig = plt.figure()
imgplot = plt.imshow(tendencie,cmap="hot")
plt.title('Tendendcies for T=16:[orange=1;black=7]')

X: (252, 64)  y:  (252,) T: 54
(54,) (64,) (252, 54)
(252, 64)
(252, 54)


ValueError: operands could not be broadcast together with shapes (109,64) (54,) 